In [29]:
import os
import sys
module_path = os.path.abspath(os.path.join('..\\PythonModules\\'))                                           
if module_path not in sys.path:
    sys.path.append(module_path)
##import Bedrock_Client
##import pyodbc
import pandas as pd
import PredictorOptimize

In [30]:
##df = pd.read_sql_query('select * fROM ref.SHMI_Model_Predict_Statistics',Bedrock_Conn)
import Pickle_Utility
df =Pickle_Utility.read('shimi_predictor')


df.to_csv('shmi_predictor.csv')

In [87]:
import matplotlib.pyplot as plt
PREDICTOR = 'AGE_GROUP_14'
DIAGNOSIS_GROUP = '2'

new_df = df[(df['PREDICTOR']==PREDICTOR)&(df['DIAGNOSIS_GROUP']==DIAGNOSIS_GROUP)][['Date','PARAMETER_ESTIMATE']].sort_values(by='Date')


In [88]:
new_df.set_index('Date', inplace=True)

In [89]:
new_df.astype('float')

,PARAMETER_ESTIMATE
Date,
2019-05-13,-0.8306
2019-06-13,-0.8145
2019-07-13,-0.8098
2019-08-13,-0.8014
2019-09-13,-0.7887
2019-10-13,-0.7856
2019-11-13,-0.7801
2019-12-13,-0.7803
2020-01-13,-0.7784


In [90]:
from statsmodels.tsa.ar_model import AR
from statsmodels.tsa.ar_model import ARResults
import numpy
 
# create a difference transform of the dataset
def difference(dataset):
    diff = list()
    for i in range(1, len(dataset)):
        value = dataset[i] - dataset[i - 1]
        diff.append(value)
    return numpy.array(diff)
df_train = new_df.drop(new_df.tail(1).index, inplace=False) 
# load dataset
series = df_train.astype('float')
X = difference(series.values)
# fit model
model = AR(X)
model_fit = model.fit(maxlag=3, disp=False)
# save model to file
model_fit.save('ar_model.pkl')
# save the differenced dataset
numpy.save('ar_data.npy', X)
# save the last ob
numpy.save('ar_obs.npy', [series.values[-1]])

In [91]:
print(sum(X)/len(X))

[0.00682222]


In [92]:
[series.values[-1]]

[array([-0.7692])]

In [93]:
# load the AR model from file
from statsmodels.tsa.ar_model import ARResults
import numpy
loaded = ARResults.load('ar_model.pkl')
print(loaded.params)
data = numpy.load('ar_data.npy')
last_ob = numpy.load('ar_obs.npy')
print(last_ob)

[ 0.00322733  0.01049784 -0.16535987  0.3596375 ]
[[-0.7692]]


In [94]:
# load AR model from file and make a one-step prediction
from statsmodels.tsa.ar_model import ARResults
import numpy
# load model
model = ARResults.load('ar_model.pkl')
data = numpy.load('ar_data.npy')
last_ob = numpy.load('ar_obs.npy')
# make prediction
predictions = model.predict(start=len(data), end=len(data))
# transform prediction
yhat = predictions[0] + last_ob[0]
print('Prediction: %f' % yhat)

Prediction: -0.766262


In [95]:
new_df.tail(1)

,PARAMETER_ESTIMATE
Date,
2020-03-13,-0.7638


In [96]:
# AR example
from statsmodels.tsa.ar_model import AR
from random import random
# contrived dataset
data = df_train.astype('float')
# fit model
model = AR(data)
model_fit = model.fit()
# make prediction
yhat = model_fit.predict(len(data), len(data))
print(yhat)

10   -0.763953
dtype: float64
f:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
f:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:576: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  ValueWarning)


In [97]:
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from random import random
# contrived dataset
data = df_train.astype('float')
# fit model
model = SimpleExpSmoothing(data)
model_fit = model.fit()
# make prediction
yhat = model_fit.predict(len(data), len(data))
print(yhat)

10   -0.769377
dtype: float64
f:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
f:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:576: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  ValueWarning)


In [98]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from random import random
# contrived dataset
data = df_train.astype('float')
# fit model
model = ExponentialSmoothing(data)
model_fit = model.fit()
# make prediction
yhat = model_fit.predict(len(data), len(data))
print(yhat)

10   -0.769377
dtype: float64
f:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:219: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  ' ignored when e.g. forecasting.', ValueWarning)
f:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:576: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  ValueWarning)


In [99]:
data

,PARAMETER_ESTIMATE
Date,
2019-05-13,-0.8306
2019-06-13,-0.8145
2019-07-13,-0.8098
2019-08-13,-0.8014
2019-09-13,-0.7887
2019-10-13,-0.7856
2019-11-13,-0.7801
2019-12-13,-0.7803
2020-01-13,-0.7784


In [100]:
data = df_train.astype('float').values
	# data split
n_test = 3
# model configs
cfg_list = PredictorOptimize.exp_smoothing_configs(seasonal=[None])
# grid search

#walk_forward_validation(data, n_test, cfg)

scores = PredictorOptimize.grid_search(data[:,0], cfg_list, n_test)
print('done')
# list top 3 configs
for cfg, error in scores[:3]:
    print(cfg, error)

> Model[['add', True, None, None, False, True]] 0.104
 > Model[['add', True, None, None, False, False]] 0.005
 > Model[['add', False, None, None, False, True]] 0.027
 > Model[['add', False, None, None, False, False]] 0.024
 > Model[[None, False, None, None, False, True]] 0.091
 > Model[[None, False, None, None, False, False]] 0.005
done
[None, False, None, None, False, False] 0.005444364649126767
['add', True, None, None, False, False] 0.005444364768165183
['add', False, None, None, False, False] 0.023887602029880394


In [101]:
data = new_df.astype('float').values
	# data split
n_test = 3
# model configs
cfg_list = PredictorOptimize.exp_smoothing_configs(seasonal=[None])
# grid search

#walk_forward_validation(data, n_test, cfg)

scores = PredictorOptimize.grid_search(data[:,0], cfg_list, n_test)
print('done')
# list top 3 configs
for cfg, error in scores[:3]:
    print(cfg, error)

> Model[['add', True, None, None, False, True]] 0.095
 > Model[['add', True, None, None, False, False]] 0.006
 > Model[['add', False, None, None, False, True]] 0.049
 > Model[['add', False, None, None, False, False]] 0.010
 > Model[[None, False, None, None, False, True]] 0.084
 > Model[[None, False, None, None, False, False]] 0.006
done
[None, False, None, None, False, False] 0.006325008023340217
['add', True, None, None, False, False] 0.006325008617063463
['add', False, None, None, False, False] 0.01033400044804482


In [102]:
sum(data)/len(data)

array([-0.79112727])

In [103]:
PredictorOptimize.exp_smoothing_forecast(data, ['additive', True, 2, None, False, False])

-0.7639064000238105

In [104]:
data

array([[-0.8306],
       [-0.8145],
       [-0.8098],
       [-0.8014],
       [-0.7887],
       [-0.7856],
       [-0.7801],
       [-0.7803],
       [-0.7784],
       [-0.7692],
       [-0.7638]])

In [105]:
new_df

,PARAMETER_ESTIMATE
Date,
2019-05-13,-0.8306
2019-06-13,-0.8145
2019-07-13,-0.8098
2019-08-13,-0.8014
2019-09-13,-0.7887
2019-10-13,-0.7856
2019-11-13,-0.7801
2019-12-13,-0.7803
2020-01-13,-0.7784


In [106]:
import numpy as np
b=np.array([[PredictorOptimize.exp_smoothing_forecast(data, ['add', True, None, None, False, False])]])
data = np.concatenate((data, b))
#add([exp_smoothing_forecast(data, ['add', True, None, None, False, False])])
data

array([[-0.8306   ],
       [-0.8145   ],
       [-0.8098   ],
       [-0.8014   ],
       [-0.7887   ],
       [-0.7856   ],
       [-0.7801   ],
       [-0.7803   ],
       [-0.7784   ],
       [-0.7692   ],
       [-0.7638   ],
       [-0.7639064]])

In [107]:
PredictorOptimize.exp_smoothing_forecast(data, ['add', True, None, None, False, False])

-0.7639064000200587

In [108]:
config = scores[:1][0][0]
config

'[None, False, None, None, False, False]'

In [109]:
import ast
n_test = 3
def predict_n_steps(data, n_steps, use_new_model = True):
    data_history = data
    cfg_list = PredictorOptimize.exp_smoothing_configs(seasonal=[0,6,12])
    scores = PredictorOptimize.grid_search(data[:,0], cfg_list, n_test)
    config = scores[:1][0][0]
    if use_new_model == True:
        for i in range(n_steps):
            new = np.array([[PredictorOptimize.exp_smoothing_forecast(data_history,ast.literal_eval(config))]])
            data_history = np.concatenate((data_history, new))
    else:
        t,d,s,p,b,r = ast.literal_eval(config)
        model = ExponentialSmoothing(data_history, trend=t, damped=d, seasonal=s, seasonal_periods=p)
        model_fit = model.fit(optimized=True, use_boxcox=b, remove_bias=r)
        for i in range(n_steps):            
            yhat = np.array([model_fit.predict(len(data_history), len(data_history))])            
            data_history = np.concatenate((data_history, yhat))            
    return data_history

In [110]:
predict_n_steps(data,4)

> Model[['add', True, None, 0, False, True]] 0.085
 > Model[['add', True, None, 0, False, False]] 0.006
 > Model[['add', True, None, 6, False, True]] 0.085
 > Model[['add', True, None, 6, False, False]] 0.006
 > Model[['add', True, None, 12, False, True]] 0.085
 > Model[['add', True, None, 12, False, False]] 0.006
 > Model[['add', False, None, 0, False, True]] 0.064
 > Model[['add', False, None, 0, False, False]] 0.004
 > Model[['add', False, None, 6, False, True]] 0.064
 > Model[['add', False, None, 6, False, False]] 0.004
 > Model[['add', False, None, 12, False, True]] 0.064
 > Model[['add', False, None, 12, False, False]] 0.004
 > Model[[None, False, 'add', 6, False, True]] 0.145
 > Model[[None, False, 'add', 6, False, False]] 0.146
 > Model[[None, False, None, 0, False, True]] 0.076
 > Model[[None, False, None, 0, False, False]] 0.006
 > Model[[None, False, None, 6, False, True]] 0.076
 > Model[[None, False, None, 6, False, False]] 0.006
 > Model[[None, False, None, 12, False, True

array([[-0.8306   ],
       [-0.8145   ],
       [-0.8098   ],
       [-0.8014   ],
       [-0.7887   ],
       [-0.7856   ],
       [-0.7801   ],
       [-0.7803   ],
       [-0.7784   ],
       [-0.7692   ],
       [-0.7638   ],
       [-0.7639064],
       [-0.7639064],
       [-0.7639064],
       [-0.7639064],
       [-0.7639064]])

In [111]:
predict_n_steps(data,4,False)

> Model[['add', True, None, 0, False, True]] 0.085
 > Model[['add', True, None, 0, False, False]] 0.006
 > Model[['add', True, None, 6, False, True]] 0.085
 > Model[['add', True, None, 6, False, False]] 0.006
 > Model[['add', True, None, 12, False, True]] 0.085
 > Model[['add', True, None, 12, False, False]] 0.006
 > Model[['add', False, None, 0, False, True]] 0.064
 > Model[['add', False, None, 0, False, False]] 0.004
 > Model[['add', False, None, 6, False, True]] 0.064
 > Model[['add', False, None, 6, False, False]] 0.004
 > Model[['add', False, None, 12, False, True]] 0.064
 > Model[['add', False, None, 12, False, False]] 0.004
 > Model[[None, False, 'add', 6, False, True]] 0.145
 > Model[[None, False, 'add', 6, False, False]] 0.146
 > Model[[None, False, None, 0, False, True]] 0.076
 > Model[[None, False, None, 0, False, False]] 0.006
 > Model[[None, False, None, 6, False, True]] 0.076
 > Model[[None, False, None, 6, False, False]] 0.006
 > Model[[None, False, None, 12, False, True

array([[-0.8306   ],
       [-0.8145   ],
       [-0.8098   ],
       [-0.8014   ],
       [-0.7887   ],
       [-0.7856   ],
       [-0.7801   ],
       [-0.7803   ],
       [-0.7784   ],
       [-0.7692   ],
       [-0.7638   ],
       [-0.7639064],
       [-0.7639064],
       [-0.7639064],
       [-0.7639064],
       [-0.7639064]])